In [1]:
import ee, datetime
import pandas as pd
import numpy as np
import folium

import datetime as dt
import IPython.display

In [2]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [3]:
lower = folium.Map(location=[23.248990359705306, 77.4063907856056], zoom_start=10)

In [4]:
sentinel3 = ee.ImageCollection("COPERNICUS/S3/OLCI").\
              filterDate("2019-01-01","2019-06-30")
lower_AOI = ee.Geometry.Polygon([[77.40540829355803,23.250265289120904],[77.40497914011564,23.250107568377107],[77.40454998667326,23.24987098691169],[77.40394917185392,23.249397822721765],[77.40296211893644,23.249634405026583],[77.40188923533049,23.250068138162035],[77.40124550516691,23.25014699858056],[77.40030136759368,23.25006813816203],[77.39922848398771,23.250738450233026],[77.39828434641447,23.250975030159537],[77.3976406162509,23.25046243978831],[77.39802685434904,23.249910417185077],[77.39879933054533,23.249358392296827],[77.39957180674162,23.249042948477634],[77.40017262156096,23.248688073289212],[77.40068760569181,23.248490919998734],[77.40128842051116,23.248057181733795],[77.40214672739592,23.247584011108767],[77.40261879618254,23.247307994135603],[77.40334835703459,23.247071407703988],[77.40407791788664,23.2469136831831],[77.4051508014926,23.246598233581782],[77.40480747873869,23.245888469251092],[77.40429249460783,23.245415290932417],[77.40356293375578,23.245060406091877],[77.40274754221525,23.244390065483675],[77.40214672739592,23.244035177915574],[77.40150299723234,23.24344369653732],[77.40154591257658,23.242852212536526],[77.40227547342865,23.243049374161522],[77.40283337290373,23.243443696537323],[77.40334835703459,23.24360142516129],[77.40407791788664,23.24387744980447],[77.40472164805021,23.244390065483675],[77.40523663218109,23.244666088495027],[77.40588036234465,23.24490267919301],[77.40635243113127,23.24569131182231],[77.40738239939299,23.246203920530995],[77.40798321421232,23.246203920530995],[77.40849819834318,23.24608562638846],[77.40858402903166,23.247426287194042],[77.40901318247404,23.248214904902426],[77.40944233591642,23.248964087406257],[77.41000023539152,23.25006813816203],[77.41072979624357,23.250608695872465],[77.41012898142424,23.250805846032197],[77.41060105021086,23.25139729476258],[77.41111603434172,23.25171273301285],[77.4121030872592,23.251633873520237],[77.41287556345549,23.25151558419388],[77.41381970102873,23.251160715585222],[77.41377678568449,23.25147615439511],[77.41364803965178,23.252028170516947],[77.41326180155363,23.252580184353608],[77.4126609867343,23.252974478552105],[77.41177199151964,23.253329342333874],[77.41155741479845,23.25376306345126],[77.41108534601183,23.25407849610504],[77.41065619256945,23.25415735415188],[77.40958330896349,23.254630501453526],[77.4090254094884,23.25447278587288],[77.40889666345568,23.254196783157816],[77.40859625604601,23.253684205171215],[77.40812418725939,23.253171625214122],[77.40773794916124,23.252737902172907],[77.40726588037462,23.252383036817132],[77.40675089624376,23.25194931121088],[77.40615008142443,23.251633873520237],[77.40567801263781,23.251279005226362],[77.40584967401476,23.250529835726965],[77.40540829355803,23.250265289120904]])
sentinel3_AOI = sentinel3.filterBounds(lower_AOI)

In [5]:
print('Total number of images :', sentinel3_AOI.size().getInfo())

Total number of images : 952


In [6]:
def addDM(image):
  rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
  dm = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
  return image.addBands(dm)
with_dm = sentinel3_AOI.map(addDM)

In [7]:
def meanDM(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['dom']),
                           geometry = lower_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('dom').getInfo()

In [8]:
Images_dm = with_dm.select('dom').toList(with_dm.size())
dm_coll= []
for i in range(Images_dm.length().getInfo()):
  image = ee.Image(Images_dm.get(i-1))
  temp_dm = meanDM(image)
  dm_coll.append(temp_dm)

In [ ]:
dates = np.array(with_dm.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [ ]:
df6 = pd.DataFrame(dm_coll, index = day, columns = ['Dissolved Oxygen Matter'])
df6.index = pd.to_datetime(df6.index, format="%Y/%m/%d")
df6.sort_index(ascending = True, inplace = True)

In [ ]:
df6

In [ ]:
df6.to_csv('data/DOM.csv')

In [ ]:
sentinel3 = ee.ImageCollection("COPERNICUS/S3/OLCI").\
              filterDate("2018-01-01","2023-01-01")
lower_AOI = ee.Geometry.Polygon([[77.40540829355803,23.250265289120904],[77.40497914011564,23.250107568377107],[77.40454998667326,23.24987098691169],[77.40394917185392,23.249397822721765],[77.40296211893644,23.249634405026583],[77.40188923533049,23.250068138162035],[77.40124550516691,23.25014699858056],[77.40030136759368,23.25006813816203],[77.39922848398771,23.250738450233026],[77.39828434641447,23.250975030159537],[77.3976406162509,23.25046243978831],[77.39802685434904,23.249910417185077],[77.39879933054533,23.249358392296827],[77.39957180674162,23.249042948477634],[77.40017262156096,23.248688073289212],[77.40068760569181,23.248490919998734],[77.40128842051116,23.248057181733795],[77.40214672739592,23.247584011108767],[77.40261879618254,23.247307994135603],[77.40334835703459,23.247071407703988],[77.40407791788664,23.2469136831831],[77.4051508014926,23.246598233581782],[77.40480747873869,23.245888469251092],[77.40429249460783,23.245415290932417],[77.40356293375578,23.245060406091877],[77.40274754221525,23.244390065483675],[77.40214672739592,23.244035177915574],[77.40150299723234,23.24344369653732],[77.40154591257658,23.242852212536526],[77.40227547342865,23.243049374161522],[77.40283337290373,23.243443696537323],[77.40334835703459,23.24360142516129],[77.40407791788664,23.24387744980447],[77.40472164805021,23.244390065483675],[77.40523663218109,23.244666088495027],[77.40588036234465,23.24490267919301],[77.40635243113127,23.24569131182231],[77.40738239939299,23.246203920530995],[77.40798321421232,23.246203920530995],[77.40849819834318,23.24608562638846],[77.40858402903166,23.247426287194042],[77.40901318247404,23.248214904902426],[77.40944233591642,23.248964087406257],[77.41000023539152,23.25006813816203],[77.41072979624357,23.250608695872465],[77.41012898142424,23.250805846032197],[77.41060105021086,23.25139729476258],[77.41111603434172,23.25171273301285],[77.4121030872592,23.251633873520237],[77.41287556345549,23.25151558419388],[77.41381970102873,23.251160715585222],[77.41377678568449,23.25147615439511],[77.41364803965178,23.252028170516947],[77.41326180155363,23.252580184353608],[77.4126609867343,23.252974478552105],[77.41177199151964,23.253329342333874],[77.41155741479845,23.25376306345126],[77.41108534601183,23.25407849610504],[77.41065619256945,23.25415735415188],[77.40958330896349,23.254630501453526],[77.4090254094884,23.25447278587288],[77.40889666345568,23.254196783157816],[77.40859625604601,23.253684205171215],[77.40812418725939,23.253171625214122],[77.40773794916124,23.252737902172907],[77.40726588037462,23.252383036817132],[77.40675089624376,23.25194931121088],[77.40615008142443,23.251633873520237],[77.40567801263781,23.251279005226362],[77.40584967401476,23.250529835726965],[77.40540829355803,23.250265289120904]])
sentinel3_AOI = sentinel3.filterBounds(lower_AOI)

In [ ]:
print('Total number of images :', sentinel3_AOI.size().getInfo())

In [ ]:
def addSM(image):
  rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
  suspended_matter = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')
  return image.addBands(suspended_matter)
with_sm = sentinel3_AOI.map(addSM)

In [ ]:
def meanSM(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['suspended_matter']),
                           geometry = lower_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('suspended_matter').getInfo()

In [ ]:
Images_sm = with_sm.select('suspended_matter').toList(with_sm.size())
sm_coll= []
for i in range(Images_sm.length().getInfo()):
  image = ee.Image(Images_sm.get(i-1))
  temp_sm = meanSM(image)
  sm_coll.append(temp_sm)

In [ ]:
dates = np.array(with_sm.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [ ]:
df7 = pd.DataFrame(sm_coll, index = day, columns = ['Suspended Matter'])
df7.index = pd.to_datetime(df7.index, format="%Y/%m/%d")
df7.sort_index(ascending = True, inplace = True)

In [ ]:
df7

In [ ]:
df7.to_csv('data/SM.csv')